<a href="https://colab.research.google.com/github/koya-jp/AA-google-colab-kohya/blob/master/Diffusers_ControlNet.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## **Diffusers ライブラリを用いて、画像を生成するスクリプト。**
参考 ：
[日本語](https://blog.shikoan.com/controlnet_lora/#ControlNet%EF%BC%88%E3%83%9D%E3%83%BC%E3%82%BA%EF%BC%89%E3%82%92%E4%BD%BF%E3%81%86)  ,
[英語](https://huggingface.co/blog/controlnet) ,
[Github - AA-google-colab-kohya](https://github.com/koya-jp/AA-google-colab-kohya/blob/master/Diffusers_S2D2.ipynb)

In [ ]:
#@title Driveに接続, ライブラリの追加 { display-mode: "form" }

# Driveに接続
from google.colab import drive
drive.mount('/content/drive', force_remount=True)

# ライブラリの追加
# 最新版（0.18.0）
!pip install git+https://github.com/huggingface/diffusers >/dev/null 2>&1
# diffusers==0.17.1
!pip install --upgrade transformers accelerate scipy ftfy safetensors txt2img >/dev/null 2>&1
!pip install k-diffusion >/dev/null 2>&1
!pip install --upgrade xformers git+https://github.com/huggingface/accelerate.git >/dev/null 2>&1
!pip install opencv-contrib-python >/dev/null 2>&1
!pip install controlnet_aux >/dev/null 2>&1



In [ ]:
#@title 自動切断されないようにするコード { display-mode: "form" }
%%javascript
function ClickConnect(){
console.log("Working");
document.querySelector("colab-toolbar-button#connect").click()
}setInterval(ClickConnect,60000)

In [15]:
# @markdown ***

#@title ControlNet の準備 { display-mode: "form" }

# ワイルドカードを使ってファイルパスを検索する
import glob
import os

# diffusers ライブラリをインポートする
from diffusers import (StableDiffusionControlNetPipeline, ControlNetModel, UniPCMultistepScheduler, StableDiffusionImg2ImgPipeline) # + VAEModel (0.18.0)
from diffusers.utils import numpy_to_pil
from diffusers.pipelines.stable_diffusion import StableDiffusionPipelineOutput

# controlnet_auxというポーズ検出のライブラリをインポートする
from controlnet_aux import OpenposeDetector

# PyTorchという深層学習フレームワークをインポートする
import torch

# PILという画像処理ライブラリをインポートする
from PIL import Image

# JPEGファイルのパスのリストを取得
# @markdown ***ポージング学習用フォルダ***
dir = "/content/drive/MyDrive/StableDiffusion/ControlNet/spread_legs" #@param {type: "string"}
extension = "*.png" #@param {type: "string"}
images_path = os.path.join(dir, extension)
jpeg_files = glob.glob(images_path)

# ポーズのリストを作成する
poses = []
isOpenpose = True # @param {type:"boolean"}
if isOpenpose:
  # ポーズを検出する
  controlnet_check_path = "lllyasviel/ControlNet" #@param {type: "string"}
  pose_detector = OpenposeDetector.from_pretrained(controlnet_check_path) # 事前学習済みのモデルを読み込む
  for file in jpeg_files:
      # ファイルパスから画像を読み込んでポーズを検出する
      p = pose_detector(Image.open(file))
      # ポーズをリストに追加する
      poses.append(p)

# コントロールネットワークを読み込む lllyasviel/sd-controlnet-openpose
controlnet_model_path = "lpw/stable_diffusion"  #@param {type: "string"}
# 事前学習済みのモデルを読み込む。データ型はfloat16に指定する
controlnet = ControlNetModel.from_pretrained(controlnet_model_path, torch_dtype=torch.float16)

# @markdown ***


In [ ]:
# @markdown ***

# @title Model、LoRA を StableDiffusionControlNetPipeline に設定  ※LoRA の対応がまだ　{ display-mode: "form" }

#モデル
# @markdown 👚 Model
model_path = "emilianJR/majicMIX_realistic_v6" #@param {type:"string"}

device="cuda"

pipe = StableDiffusionControlNetPipeline.from_pretrained(
    model_path,
    controlnet=controlnet,
    torch_dtype=torch.float16,
    # custom_pipeline="lpw_stable_diffusion",
)

# VAE を読み込む
# vae_path = "stabilityai/sd-vae-ft-mse"  #@param {type:"string"}
# vae = VAEModel.from_pretrained(vae_path)

# LoRAを追加 /content/drive/MyDrive/StableDiffusion/Lora/japaneseDollLikeness_v15.safetensors
# lora_path = "Kanbara/doll-likeness-series/japaneseDollLikeness_v15.safetensors" #@param {type:"string"}
# pipe.unet.load_attn_procs(lora_path)

# スケジューラーを設定
pipe.scheduler = UniPCMultistepScheduler.from_config(pipe.scheduler.config)

# @markdown 👙 NSFWを表示する設定
# Disabling safety checker
isSafeNsfwOn = True  #@param {type:"boolean"}
if isSafeNsfwOn == True:
    pipe.safety_checker = lambda images, **kwargs: (images, [False] * len(images))
else:
    pipe.safety_checker = lambda images, **kwargs: (images, [True] * len(images))

pipe.to(device)

# @markdown ***

In [21]:
# @title 画像生成を実行 { display-mode: "form" }

# 必要なモジュールやライブラリをインポートする
import os
import random
import datetime

from PIL.PngImagePlugin import PngInfo

# ファイル名に使う日付と時刻のフォーマットを定義する
file_format = "%Y%m%d_%H%M%S"
i=0

# 現在の日本時間を取得
jst_now = datetime.datetime.now(datetime.timezone(datetime.timedelta(hours=9)))

# @markdown ***
#txt2img出力画像の保存先
#@markdown ***📂 出力画像を保存するフォルダ***
save_path = "/content/drive/MyDrive/StableDiffusion/txt2img_output/bikini_controlnet/" #@param {type: "string"}
os.makedirs(save_path, exist_ok=True)
# @markdown ***

#@@markdown ポジティブプロンプト
prompt = "Beautiful girl in bikini, sitting, (spreading legs:1.5),(very wet body, clothes are very soaked:1.2),(masterpiece),((super detailed)), realistic8K UHD,(best quality:1. 2), high definition, exquisite details, fine texture, high detail, fine beautiful delicate eyes, perfectly proportioned face, light particles, distinct_ image, high resolution, high quality textures and shadows, realistic and beautiful face with big eyes, blush, glossy lips and perfect proportions, depth of field, lens flare, ray tracing, perspective, prominent nose, slender face, perfectly toned body, (thin waist:1. 3), medium chest, (glossy hands),(glossy skin), (pureerosface_v1:0.5), braun short hair, ((low angle,camel toes)), (exposed bikini pantus:1.4)BREAK (smiling, embarrassed), nsfw ,in beach" #@param {type:"string"}
#@@markdown ネガティブプロンプト
negative = "(ng_deepnegative_v1_75t),(negative_hand-neg:1.2),painting, sketch, (worst picture quality:2), (low quality:2), (normal quality:2), bad feet, lowres, ((monochrome)),skin blemishes, acne, skin blemishes, bad anatomy, (bad fingers, bad hands, missing fingers:1.3), text, error, missing fingers, extra fingers, few digits, crop, worst picture quality, jpeg image, signature, watermark, Username, blurred, bad feet, (mutation, deformity:1. 3), extra limbs, fused fingers, long neck, crossed eyes, very low resolution, bad proportions" #@param {type:"string"}
#@markdown ***🖼 出力枚数***
batch_count = 200 #@param {type: "integer"}
#@markdown ***🦵 ステップ数***
steps = 60 #@param {type:"integer"}
#@@markdown 画像サイズ
img_width = "1024" # @param [512, 768, 1024, 1536]
img_height = "1536" # @param [512, 768, 1024, 1536]
#@@markdown CFG
CFG = 8 #@param {type: "number"}
#@@markdown シード（-1の時はランダム）
seed = -1 #@param {type: "integer"}
if seed is None or seed == -1:
  inputSeed = random.randint(0, 2147483647)
else:
  valueSeed = seed

# run stable diffusion
images = []
generator = torch.Generator(device)

for i in range(batch_count):

  if seed is None or seed == -1: valueSeed = inputSeed + i
  poseIndex = random.randint(0, len(poses) - 1)
  generator.manual_seed(valueSeed)

  image = pipe(
    prompt,
    poses[poseIndex],
    negative_prompt=negative,
    generator=generator,
    num_inference_steps=steps,
    guidance_scale=CFG,
    width=int(img_width),
    height=int(img_height),
    output_type="pil",
  )

  # 現在の日本時間を取得
  jst_now = datetime.datetime.now(datetime.timezone(datetime.timedelta(hours=9)))
  #出力する画像の名前を生成する
  file_name = (jst_now.strftime(file_format)+ "_" + str(valueSeed))
  image_name = file_name + f".png"

  #画像を保存する
  save_location = os.path.join(save_path, image_name)

  # 生成された画像のリストを取得
  images = image.images

  # 生成された画像の数を取得
  num_images = len(images)

  # 生成された画像を順番に保存
  for i in range(num_images):
    # 画像を取り出す
    image = images[i]

    # @markdown ***
    #@markdown ***✎ メタデータの書き込み***
    save_metadata = True #@param {type: "boolean"}
    if save_metadata:
      metadata = PngInfo()
      metadata.add_text("prompt",(prompt))
      metadata.add_text("negative",(negative))
      metadata.add_text("steps",(str(steps)))
      metadata.add_text("CFG",(str(CFG)))
      metadata.add_text("width",(str(img_width)))
      metadata.add_text("height",(str(img_height)))
      metadata.add_text("seed",str((valueSeed)))
      image.save(save_location, pnginfo=metadata)

    # dst = Image.new('RGB', (image.shape[1], image.shape[0]))
    # dst.paste(output, (0, 0))

    # # 画像保存。googleドライブの出力ディレクトリに、年月日時分秒からなるファイル名で保存
    # filename = datetime.datetime.now(datetime.timezone(datetime.timedelta(hours=9))).strftime('%Y%m%d%H%M%S')
    # dst.save(out_dir+'/'+filename+'.png')

    # dst # 画像表示

    # @markdown ***


TypeError: ignored

In [ ]:
# @title 画像のメタデータを出力

import sys
from PIL import Image
from PIL.PngImagePlugin import PngInfo

#@markdown **保存した画像のパス**
file_dir = "/content/drive/MyDrive/StableDiffusion/txt2img_output/"  #@param {type: "string"}
file_name = "20230707_235255_101570535.png" #@param {type: "string"}
file_path = os.path.join(file_dir, file_name)
img = Image.open(file_path)

print("Prompt: ",img.text["prompt"])
print("Negative Prompt: ",img.text["negative"])
print("Scheduler: ", img.text["scheduler"])
print("Steps: ",img.text["steps"])
print("CFG: ",img.text["CFG"])
print("Width: ",img.text["width"])
print("Height: ",img.text["height"])
print("Seed: ",img.text["seed"])

try:
  print("Upscaling ratio; ",img.text["upscaling ratio"])
  print("Up steps: ",img.text["up steps"])
  print("Denoising strength: ",img.text["denoising strength"])
except:
  print("Hires.fix was OFF.")

# %%writefile output.txt